In [203]:
#%cd '/Users/pc/Desktop/Deep Learning/Project Github/CS7643_final_project'

In [204]:
#!python src/holdup/datapickler.py

# Notebook consists of two parts
# 1) Model Tuning

# 2) Generating dictionaries used for visualization

## Run code below for functions

In [271]:
from holdup.the_model.autoencoder import Autoencoder
import numpy as np
# import pandas as pd
import torch
from torch import nn
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import random
import os
from holdup.parser.replayable_hand import ReplayableHand, Streets
import functools
from typing import Tuple, List
import matplotlib.pyplot as plt
from holdup.the_model.get_datasets import *
import pandas as pd
from sklearn.model_selection import train_test_split

preflop = "preflop"
flop = "flop"
turn = "turn"
river = "river"

def get_stage(dataset, stage):
    if stage == preflop:
        return dataset[0]
    if stage == flop:
        return dataset[1]
    if stage == turn:
        return dataset[2]
    if stage == river:
        return dataset[3]

def flatten_streets(dataset):
    streets = [[], [], [], []]
    for logfile in dataset:
        for index, street in enumerate(logfile):
            streets[index] = streets[index] + street
    return streets


def get_data(dataset, stage):
    flattened_data = flatten_streets(dataset)
    stage_data = get_stage(flattened_data, stage)
    return [(x[0], x[1][1]) for x in stage_data]


with open('last_possible.pickle', 'rb') as last_possible_pickle:
    last_possible_dataset = pickle.load(last_possible_pickle)

preflop_data = get_data(last_possible_dataset, "preflop")
flop_data = get_data(last_possible_dataset, "flop")
turn_data = get_data(last_possible_dataset, "turn")
river_data = get_data(last_possible_dataset, "river")

def separate_train_test(street_data):
    n_train = int(len(street_data)*0.6)
    train_set = street_data[:n_train]
    test_set = street_data[n_train:]
    return train_set,test_set

train_preflop, test_preflop =separate_train_test(preflop_data)
print("preflop_train_data_size: {}".format(len(train_preflop)))
print("preflop_test_data_size: {}".format(len(test_preflop)))

train_flop, test_flop=separate_train_test(flop_data)
print("flop_train_data_size: {}".format(len(train_flop)))
print("flop_test_data_size: {}".format(len(test_flop)))

train_turn, test_turn=separate_train_test(turn_data)
print("turn_train_data_size: {}".format(len(train_turn)))
print("turn_test_data_size: {}".format(len(test_turn)))

train_river, test_river=separate_train_test(river_data)
print("river_train_data_size: {}".format(len(train_river)))
print("river_test_data_size: {}".format(len(test_river)))


# Set the device to use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create an instance of the autoencoder
# model = Autoencoder(num_hidden_nodes).to(device)

def train2(learning_rate, model, train_loader, num_epochs, wd, pftr='stage_name'):
    criterion = nn.CrossEntropyLoss() #changed to cross entropy loss for classification based tasks (semi-supervised)
    optimizer = optim.Adam(model.parameters(),lr=learning_rate, weight_decay=wd)
    train_losses = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for data in train_loader:
            inputs, labels = data
            inputs = inputs.float()
            optimizer.zero_grad()
            batch_size, _, _ = inputs.size()
            inputs = inputs.view(batch_size, -1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)


    
def quick_test2(model,test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.float()
            batch_size, _, _ = inputs.size()
            inputs = inputs.view(batch_size, -1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return round((correct/total)*100, 2)


def train_and_quick_test(lr, num_hidden_nodes, num_epochs, weight_decay,train_data,test_data, pftr):
    # Define the model
    model = Autoencoder(num_hidden_nodes).to(device)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, shuffle=False)
    train2(lr, model, train_loader, num_epochs, weight_decay,pftr)
    # Test the model
    quick_test(model,test_loader)
    

###

def tune_model(learning_rate, weight_decay, num_epochs, train, test, pftr, hidden_nodes, batch_size):
        lr = {}
        for i in range(len(learning_rate)):
            lr[str(learning_rate[i])] = []
            for j in range(len(weight_decay)):
                model = Autoencoder(hidden_nodes).to(device)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
                train2(learning_rate[i], model, train_loader, num_epochs, weight_decay[j],pftr)
                result = quick_test2(model,test_loader)
                lr[str(learning_rate[i])].append(result)
        return lr
    
def get_nodes_dict(num_experiments, max_nodes, learning_rate, weight_decay, num_epochs, train, test, pftr, batch_size):
        nd = {}
        for i in range(10, max_nodes+10, 10):
            nd[str(i)] = []
            for j in range(num_experiments):
                model = Autoencoder(i).to(device)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
                train2(learning_rate, model, train_loader, num_epochs, weight_decay,pftr)
                result = quick_test2(model,test_loader)
                nd[str(i)].append(result)
            nd[str(i)] = tuple(nd[str(i)])
        return nd
    

def get_epochs_dict(num_experiments, nodes, learning_rate, weight_decay, max_epochs, train, test, pftr, batch_size):
        nd = {}
        for i in range(10, max_epochs+10, 10):
            nd[str(i)] = []
            for j in range(num_experiments):
                model = Autoencoder(nodes).to(device)
                train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
                test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
                train2(learning_rate, model, train_loader, i, weight_decay,pftr)
                result = quick_test2(model,test_loader)
                nd[str(i)].append(result)
            nd[str(i)] = tuple(nd[str(i)])
        return nd


preflop_train_data_size: 30650
preflop_test_data_size: 20434
flop_train_data_size: 23450
flop_test_data_size: 15634
turn_train_data_size: 18447
turn_test_data_size: 12298
river_train_data_size: 16371
river_test_data_size: 10915


# 1) Tuning the model

## Best Model Parameters: 

### Learning Rate: .01 
### Weight Decay: 0 (as it decreases it improves test accuracy)
### Batch Size: 20 (Using a batch size of 20 as default as batch size doesn't have significant impact on model performance)

## Code below is used to see the test accuracy obtained by using different learning rates with different weight decays. It can be seen that using a learning rate of .01 gave the best performance out of all of the models and that by decreasing weight decay the model works better and for that reason we opted to using a weight decay of zero.

In [274]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

pre_flop = tune_model(lr, wd, 20, train_preflop, test_preflop, pftr='preflop_last_possible', hidden_nodes=20, batch_size=20)


In [275]:
pre_flop

print("PRE FLOP")
for key, value in pre_flop.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))
    

PRE FLOP
learning rate: 0.0001
     weight decay: 0 accuracy: 79.73
     weight decay: 0.0001 accuracy: 79.55
     weight decay: 0.1 accuracy: 45.97
learning rate: 0.001
     weight decay: 0 accuracy: 81.1
     weight decay: 0.0001 accuracy: 80.52
     weight decay: 0.1 accuracy: 45.97
learning rate: 0.01
     weight decay: 0 accuracy: 82.95
     weight decay: 0.0001 accuracy: 80.41
     weight decay: 0.1 accuracy: 45.97
learning rate: 0.1
     weight decay: 0 accuracy: 81.47
     weight decay: 0.0001 accuracy: 78.99
     weight decay: 0.1 accuracy: 30.58


In [253]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

flop = tune_model(lr, wd, 20, train_flop, test_flop, pftr='flop_last_possible', hidden_nodes=20, batch_size=20)

In [254]:
print("FLOP")
for key, value in flop.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

FLOP
learning rate: 0.0001
     weight decay: 0 accuracy: 81.83
     weight decay: 0.0001 accuracy: 82.05
     weight decay: 0.1 accuracy: 56.77
learning rate: 0.001
     weight decay: 0 accuracy: 83.8
     weight decay: 0.0001 accuracy: 83.16
     weight decay: 0.1 accuracy: 56.77
learning rate: 0.01
     weight decay: 0 accuracy: 85.93
     weight decay: 0.0001 accuracy: 83.91
     weight decay: 0.1 accuracy: 56.77
learning rate: 0.1
     weight decay: 0 accuracy: 83.19
     weight decay: 0.0001 accuracy: 77.2
     weight decay: 0.1 accuracy: 56.77


In [255]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

turn = tune_model(lr, wd, 40, train_turn, test_turn, pftr='turn_last_possible', hidden_nodes=40, batch_size=20)

In [256]:
print("TURN")
for key, value in turn.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

TURN
learning rate: 0.0001
     weight decay: 0 accuracy: 90.36
     weight decay: 0.0001 accuracy: 90.45
     weight decay: 0.1 accuracy: 68.31
learning rate: 0.001
     weight decay: 0 accuracy: 91.14
     weight decay: 0.0001 accuracy: 90.61
     weight decay: 0.1 accuracy: 68.31
learning rate: 0.01
     weight decay: 0 accuracy: 91.39
     weight decay: 0.0001 accuracy: 90.5
     weight decay: 0.1 accuracy: 68.31
learning rate: 0.1
     weight decay: 0 accuracy: 89.57
     weight decay: 0.0001 accuracy: 87.92
     weight decay: 0.1 accuracy: 68.31


In [257]:
lr = [.0001, .001, .01, .1]
wd = [0, .0001, .1]

river = tune_model(lr, wd, 40, train_river, test_river, pftr='river_last_possible', hidden_nodes=40, batch_size=20)

In [258]:
print("RIVER")
for key, value in river.items():
    print("learning rate: {}".format(key))
    for j in range(len(value)):
        print("     weight decay: {}".format(wd[j]), "accuracy: {}".format(value[j]))

RIVER
learning rate: 0.0001
     weight decay: 0 accuracy: 87.36
     weight decay: 0.0001 accuracy: 87.35
     weight decay: 0.1 accuracy: 59.06
learning rate: 0.001
     weight decay: 0 accuracy: 89.33
     weight decay: 0.0001 accuracy: 89.4
     weight decay: 0.1 accuracy: 59.06
learning rate: 0.01
     weight decay: 0 accuracy: 89.68
     weight decay: 0.0001 accuracy: 88.52
     weight decay: 0.1 accuracy: 59.06
learning rate: 0.1
     weight decay: 0 accuracy: 87.66
     weight decay: 0.0001 accuracy: 85.32
     weight decay: 0.1 accuracy: 59.06


## Code Below is used to demonstrate how changing the batch size does not make significant change to improving the test accuracy of the model


In [215]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    pre_flop = tune_model(lr, wd, 20, train_preflop, test_preflop, pftr='preflop_last_possible', hidden_nodes=20, batch_size=b_s[z])
    ls.append(pre_flop)

print("PRE FLOP")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))



PRE FLOP
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 83
     When using batch size of 16, accuracy is 83
     When using batch size of 32, accuracy is 83
     When using batch size of 64, accuracy is 83
     When using batch size of 128, accuracy is 83
     When using batch size of 256, accuracy is 83
     When using batch size of 512, accuracy is 82
     When using batch size of 1024, accuracy is 81


In [259]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    flop = tune_model(lr, wd, 20, train_flop, test_flop, pftr='flop_last_possible', hidden_nodes=20, batch_size=b_s[z])
    ls.append(flop)

print("FLOP")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


FLOP
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 85.45
     When using batch size of 16, accuracy is 85.75
     When using batch size of 32, accuracy is 86.0
     When using batch size of 64, accuracy is 85.81
     When using batch size of 128, accuracy is 85.54
     When using batch size of 256, accuracy is 85.31
     When using batch size of 512, accuracy is 84.4
     When using batch size of 1024, accuracy is 83.57


In [260]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    turn = tune_model(lr, wd, 40, train_turn, test_turn, pftr='turn_last_possible', hidden_nodes=40, batch_size=b_s[z])
    ls.append(turn)
    
print("TURN")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


TURN
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 91.67
     When using batch size of 16, accuracy is 91.38
     When using batch size of 32, accuracy is 91.76
     When using batch size of 64, accuracy is 91.63
     When using batch size of 128, accuracy is 91.67
     When using batch size of 256, accuracy is 91.8
     When using batch size of 512, accuracy is 91.46
     When using batch size of 1024, accuracy is 90.71


In [261]:
lr = [.01]
wd = [0]
b_s = [8,16,32,64,128,256,512,1024]

ls = []

for z in range(len(b_s)):
    river = tune_model(lr, wd, 40, train_river, test_river, pftr='river_last_possible', hidden_nodes=40, batch_size=b_s[z])
    ls.append(river)
    
print("RIVER")
print("Using learning rate of {} and weight decay of {}:".format(lr[0], wd[0]))
for i in range(len(ls)):
    for key, value in ls[i].items():
        print("     When using batch size of {}, accuracy is {}".format(b_s[i], value[0]))


RIVER
Using learning rate of 0.01 and weight decay of 0:
     When using batch size of 8, accuracy is 89.57
     When using batch size of 16, accuracy is 89.44
     When using batch size of 32, accuracy is 89.9
     When using batch size of 64, accuracy is 89.8
     When using batch size of 128, accuracy is 89.74
     When using batch size of 256, accuracy is 89.44
     When using batch size of 512, accuracy is 89.7
     When using batch size of 1024, accuracy is 89.62


# 2) Generating visualization dictionaries - Run all code lines below including the code to pickle the dictionaries

In [272]:
flop_nd = get_nodes_dict(10, 100, .01, 0, 20, train_flop, test_flop, pftr='flop_last_possible', batch_size=20)

In [273]:
flop_nd

{'10': (84.82, 84.74), '20': (86.05, 86.11)}

In [ ]:
turn_nd = get_nodes_dict(10, 100, .01, 0, 40, train_turn, test_turn, pftr='turn_last_possible', batch_size=20)

In [ ]:
turn_nd

In [ ]:
river_nd = get_nodes_dict(10, 100, .01, 0, 40, train_river, test_river, pftr='river_last_possible', batch_size=20)

In [ ]:
river_nd

In [245]:
flop_ed = get_epochs_dict(10, 20, .01, 0, 100, train_flop, test_flop, pftr='flop_last_possible', batch_size=20)


In [246]:
flop_ed

{'10': (85.13, 85.17), '20': (85.75, 85.74)}

In [247]:
turn_ed = get_epochs_dict(10, 40, .01, 0, 100, train_turn, test_turn, pftr='turn_last_possible', batch_size=20)

In [248]:
turn_ed

{'10': (91.15, 91.51), '20': (91.75, 91.45)}

In [251]:
river_ed = get_epochs_dict(10, 40, .01, 0, 100, train_river, test_river, pftr='river_last_possible', batch_size=20)

In [252]:
river_ed

{'10': (89.23, 89.53), '20': (89.63, 89.42)}

## Running the line below will pickle the dictionaries that will be used for the visualization

In [196]:
import pickle

with open("flop_nd.pickle", "wb") as f: 
    pickle.dump(flop_nd, f)
    
with open("turn_nd.pickle", "wb") as f: 
    pickle.dump(turn_nd, f)
    
with open("river_nd.pickle", "wb") as f: 
    pickle.dump(river_nd, f)
    
with open("flop_ed.pickle", "wb") as f: 
    pickle.dump(flop_ed, f)
    
with open("turn_ed.pickle", "wb") as f: 
    pickle.dump(turn_ed, f)
    
with open("river_ed.pickle", "wb") as f: 
    pickle.dump(river_ed, f)